In [1]:
import ipeadatapy as ipea
import pandas as pd
import numpy as np

In [2]:
series = ipea.list_series()
series.tail()

,CODE,NAME
9603,ADH_RDPC4_URB,Renda domiciliar per capita média do 4º quint...
9604,ADH_RDPC5_RUR,Renda domiciliar per capita média do quinto ma...
9605,ADH_RDPC5_URB,Renda domiciliar per capita média do quinto ma...
9606,ADH_THEIL_RUR,Índice de Theil - L - áreas rurais (Atlas DH -...
9607,ADH_THEIL_URB,Índice de Theil - L - áreas urbanas (Atlas DH ...


In [39]:
series.loc[series.NAME == "PIB - preços de mercado"]

,CODE,NAME
4428,SCN10_PIBN10,PIB - preços de mercado
4580,SCN104_PIBPMV104,PIB - preços de mercado


In [41]:
ipea.describe("SCN10_PIBN10")

,PIB - preços de mercado
Name,PIB - preços de mercado
Code,SCN10_PIBN10
Big Theme,Macroeconômico
Theme code,8
Source,Instituto Brasileiro de Geografia e Estatístic...
Source acronym,IBGE/SCN Anual
Comment,O produto interno bruto (PIB) é o total dos be...
Last update,2023-03-02T17:16:01-03:00
Frequency,Anual
Measure,R$


In [42]:
df = (
    pd.read_csv("deflator.csv")
    .rename({"DPCERG3A086NBEA": "deflator"}, axis = 1)
    .assign(DATE = lambda _: pd.to_datetime(_.DATE),
            deflator = lambda _: _.deflator / 100)
)

bases = {
    "exportacao": ipea.describe("BPN_XTV"),
    "importacao": ipea.describe("BPN_MTV"),
    "base_monetaria": ipea.describe("BM_M0FN"),
    "inflacao": ipea.describe("IGP_IGPDIG"),
    "cambio": ipea.describe("BM_ERCF"),
    "divida_externa": ipea.describe("BM_DEXR"),
    "dslp": ipea.describe("BM_DTGFN"),
    "pib_industria": ipea.describe("SCN10_VAINDN10"),
    "pib": ipea.describe("SCN10_PIBN10")
}


for base, dados in bases.items():
    bases[base] = list(dados.to_dict().values())[0]
    bases[base]["dataframe"] = (
        ipea.timeseries(bases[base]["Code"]).reset_index()
        .rename({f"VALUE ({bases[base]['Measure']})": "VALUE"}, axis = 1)
        .assign(
            nome = base,
            unidade = bases[base]['Unit'], 
            medida = bases[base]['Measure'])
    )


df = (
    pd.concat([base["dataframe"] for base in bases.values()])
    .merge(df, how = "left")
    .pipe(lambda DF: DF.merge(DF
        .query("nome == 'cambio'")
        .rename({"VALUE": "cambio"}, axis = 1)
        .filter(["DATE", "cambio"]), 
    how = "left")
    )
)

relatorio = (df
 .query("DATE.dt.year > 1950 and DATE.dt.year < 2000")
 .assign(
    unidade = lambda _: 
        np.where(_.unidade == 'milhões', 1000000,
        np.where(_.unidade.isna(), 1, np.nan)
    ),
    multiplicador = lambda _: np.where(_.nome == "cambio", 1, 
        (_.unidade) / (
            (_.deflator ** ((_.medida == "US$") | (_.medida == "R$"))) * #Deflacionar os dolares
            (_.cambio ** (_.medida == "R$")) #Trocar reais por dolares
        )
    ),
    VALUE = lambda _: _.VALUE * _.multiplicador
).pivot_table(
    index = "YEAR",
    columns = "nome",
    values = "VALUE"
).reset_index()
 .rename_axis("", axis = 1)
)

In [43]:
relatorio.to_csv("data.csv")
relatorio.head()

,YEAR,base_monetaria,cambio,divida_externa,dslp,exportacao,importacao,inflacao,pib,pib_industria
0,1951,1.679678e+10,6.683636e-15,4.033910e+09,NaN,1.249736e+10,1.218505e+10,12.337655,1.340668e+11,3.124893e+10
1,1952,1.935976e+10,6.683636e-15,4.416753e+09,NaN,9.817355e+09,1.190897e+10,12.716743,1.545015e+11,3.480238e+10
2,1953,7.635334e+09,1.981818e-14,7.919371e+09,NaN,1.051808e+10,7.820998e+09,20.512819,6.137104e+10,1.455603e+10
3,1954,6.964208e+09,2.618182e-14,8.104079e+09,NaN,1.058298e+10,9.585899e+09,25.864923,6.316725e+10,1.502951e+10
4,1955,9.187637e+09,2.345455e-14,9.415497e+09,NaN,9.606142e+09,7.449987e+09,12.145735,8.525248e+10,2.045767e+10


In [27]:
bases["divida_externa"]

{'Name': 'Dívida externa - registrada - INATIVA',
 'Code': 'BM_DEXR',
 'Big Theme': 'Macroeconômico',
 'Theme code': 10,
 'Source': 'Banco Central do Brasil, Boletim, Seção Balanço de Pagamentos (Bacen / Boletim / BP)',
 'Source acronym': 'Bacen/Boletim/BP',
 'Comment': 'Para 1889-1945: Abreu, Marcelo de Paiva (Org.). A ordem do progresso - cem anos de política econômica republicana. Rio de Janeiro: Campus, 1992. Obs.: A partir de mar. 2001, exclui empréstimos intercompanhias (retroativo a 2000) e contempla revisão na posição de endividamento. Compreende o setor público financeiro e não financeiro e o setor privado. Série interrompida.',
 'Last update': '2009-06-26T16:30:13-03:00',
 'Frequency': 'Anual',
 'Measure': 'US$',
 'Unit': 'milhões',
 'Status': 'I',
 'dataframe':           DATE     CODE                   RAW DATE  DAY  MONTH  YEAR  \
 0   1889-01-01  BM_DEXR  1889-01-01T00:00:00-02:00    1      1  1889   
 1   1890-01-01  BM_DEXR  1890-01-01T00:00:00-02:00    1      1  1890   